This notebook demonstrates the cross-validation functions to assess the label assist performance. Ouputs include confusion matrices and tables of standard performance metrics (precision, recall, f-score, accuracy). 

### imports and setup

In [ ]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np

# our imports
import emission.storage.timeseries.abstract_timeseries as esta
import emission.storage.decorations.trip_queries as esdtq
import models
from performance_eval import cross_val_predict, print_clf_metrics, get_clf_metrics, cv_for_all_users

# set up logging
import logging 
logging.getLogger().setLevel(logging.INFO)

### Read data and set up variables

Let's see how many labeled/unlabeled trips there are

In [ ]:
all_users = esta.TimeSeries.get_uuid_list()
confirmed_trip_df_map = {}
labeled_trip_df_map = {}
expanded_trip_df_map = {}
for u in all_users:
    ts = esta.TimeSeries.get_time_series(u)
    ct_df = ts.get_data_df("analysis/confirmed_trip")
    confirmed_trip_df_map[u] = ct_df
    labeled_trip_df_map[u] = esdtq.filter_labeled_trips(ct_df)
    expanded_trip_df_map[u] = esdtq.expand_userinputs(labeled_trip_df_map[u])
n_trips_df = pd.DataFrame(
    [[u, len(confirmed_trip_df_map[u]),
      len(labeled_trip_df_map[u])] for u in all_users],
    columns=["user_id", "all_trips", "labeled_trips"])
n_trips_df

In [ ]:
all_trips = n_trips_df.all_trips.sum()
labeled_trips = n_trips_df.labeled_trips.sum()
unlabeled_trips = all_trips - labeled_trips
print('{:.2f}% unlabeled, {:.2f}% labeled, {} total trips'.format(
    unlabeled_trips / all_trips, labeled_trips / all_trips, all_trips))


### get results

In [ ]:
cross_val_results = cv_for_all_users(models.ClusterExtrapolationClassifier,
                                uuid_list=all_users,
                                expanded_trip_df_map=expanded_trip_df_map,
                                raise_errors=True)

In [ ]:
print_clf_metrics(cross_val_results,
                  'mode',
                  weight='distance',
                  keep_nopred=True,
                  ignore_custom=False)

In [ ]:
# ensure that the distances in the confusion matrix match up with the actual 
# distances from the user
print(np.sum(get_clf_metrics(cross_val_results, 'mode', weight='distance',
                  keep_nopred=True,
                  ignore_custom=False)['cm']))
print(expanded_trip_df_map[user2].dropna(subset=['mode_confirm']).distance.sum())

In [ ]:
print_clf_metrics(cross_val_results,
                  'purpose',
                  weight='distance',
                  keep_nopred=True,
                  ignore_custom=False)

In [ ]:
# we can also access the standard confusion matrix based on trip count instead 
# of trip distances
print_clf_metrics(cross_val_results,
                  'purpose',
                  weight='count',
                  keep_nopred=True,
                  ignore_custom=False)

In [ ]:
print_clf_metrics(cross_val_results,
                  'replaced',
                  weight='distance',
                  keep_nopred=True,
                  ignore_custom=False)